<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml0606/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 6s 0us/step


In [2]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [3]:
import os
import glob
dn = os.path.dirname(dataset)
glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")

['/root/.keras/datasets/aclImdb/train/pos/11452_8.txt',
 '/root/.keras/datasets/aclImdb/train/pos/8553_7.txt',
 '/root/.keras/datasets/aclImdb/train/pos/5005_8.txt',
 '/root/.keras/datasets/aclImdb/train/pos/2826_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/7151_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/670_7.txt',
 '/root/.keras/datasets/aclImdb/train/pos/748_9.txt',
 '/root/.keras/datasets/aclImdb/train/pos/7205_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/11030_8.txt',
 '/root/.keras/datasets/aclImdb/train/pos/2336_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/3580_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/3296_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/5613_9.txt',
 '/root/.keras/datasets/aclImdb/train/pos/8033_7.txt',
 '/root/.keras/datasets/aclImdb/train/pos/11187_9.txt',
 '/root/.keras/datasets/aclImdb/train/pos/8366_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/5324_10.txt',
 '/root/.keras/datasets/aclImdb/train/pos/12481_8.txt',


In [4]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [5]:
train_df = getdata("train")
test_df = getdata("test")

In [6]:
test_df

,content,sentiment
0,"Anatole Litvak directed the 1959 film, ""The Jo...",1
1,Talkshow with Spike Feresten is one of those s...,1
2,when I know that Walter will never grace anoth...,1
3,One of the most sublime of American masterpiec...,1
4,I saw this film at the Sundance Film Festival ...,1
...,...,...
24995,Good lord.<br /><br />I'm going to say right o...,0
24996,Margaret Mitchell spins in her grave every tim...,0
24997,"This movie is very violent, yet exciting with ...",0
24998,This movie has been promoting in everywhere in...,0


In [8]:
# 1. Tokenize: I love you -> 1 2 3
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [11]:
# tok.word_index
# tok.index_word
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [ ]:
pd.DataFrame(x_train_seq)

In [17]:
# 2. padding: 把所有序列變成一樣長
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)

In [18]:
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,5,27,255,8,1,154,4,11,1730,1737
1,0,0,0,0,0,0,0,0,0,0,...,10,516,9,690,155,40,15,109,157,4
2,0,0,0,0,0,0,0,0,0,0,...,2,42,1,636,12,23,47,15,1,1467
3,0,0,0,0,0,0,0,0,0,0,...,44,22,123,25,1,576,78,21,714,9
4,0,0,0,0,0,0,0,0,0,0,...,17,141,27,20,126,347,454,5,64,1025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,5,295,26,249,263,122,14,309,2,2
24996,0,0,0,0,0,0,0,0,0,0,...,1903,745,163,1,19,430,21,1,750,111
24997,0,0,0,0,0,0,0,0,0,0,...,6,2402,12,33,77,27,915,5,1773,965
24998,0,0,0,0,0,0,0,0,0,0,...,2,1035,79,1680,969,5,3,165,43,959


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

layers = [
    # 3001種 * 100 -> 300100
    Embedding(3001, 100, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 100)          300100    
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 300,302
Trainable params: 300,302
Non-trainable params: 0
_________________________________________________________________


In [20]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [21]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [22]:
# batch_size: 你看多少筆再進行一次梯度下降(5[圖片大/訓練資料少]-200[圖片小/訓練資料多])
# epochs: 全體資料(60000) 你要看幾輪(我們什麼時候要停的關鍵)
# validation_split: 測試什麼時候停下來要用沒看過的資料(val loss穩定的時候就停, 不要過擬合)
# (60000 * 0.9) / 200 -> 270
# verbose: 0(quiet) 1(full) 2(no progress bar)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    ModelCheckpoint("imdb.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=200,
     epochs=50,
     validation_split=0.1,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 7s - loss: 0.6448 - accuracy: 0.6175 - val_loss: 0.6431 - val_accuracy: 0.6556 - 7s/epoch - 63ms/step
Epoch 2/50
113/113 - 6s - loss: 0.4938 - accuracy: 0.8116 - val_loss: 0.5196 - val_accuracy: 0.7644 - 6s/epoch - 54ms/step
Epoch 3/50
113/113 - 6s - loss: 0.3847 - accuracy: 0.8567 - val_loss: 0.4289 - val_accuracy: 0.8100 - 6s/epoch - 52ms/step
Epoch 4/50
113/113 - 6s - loss: 0.3297 - accuracy: 0.8751 - val_loss: 0.3611 - val_accuracy: 0.8484 - 6s/epoch - 57ms/step
Epoch 5/50
113/113 - 8s - loss: 0.2969 - accuracy: 0.8864 - val_loss: 0.3772 - val_accuracy: 0.8296 - 8s/epoch - 70ms/step
Epoch 6/50
113/113 - 6s - loss: 0.2753 - accuracy: 0.8941 - val_loss: 0.3711 - val_accuracy: 0.8372 - 6s/epoch - 50ms/step
Epoch 7/50
113/113 - 6s - loss: 0.2595 - accuracy: 0.9009 - val_loss: 0.3072 - val_accuracy: 0.8732 - 6s/epoch - 49ms/step
Epoch 8/50
113/113 - 5s - loss: 0.2482 - accuracy: 0.9056 - val_loss: 0.3320 - val_accuracy: 0.8588 - 5s/epoch - 46ms/step
Epoch 9/50
113/1

In [24]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.2962 - accuracy: 0.8790


[0.2961641252040863, 0.8789600133895874]

In [28]:
comment = input("comment:")
seq = tok.texts_to_sequences([comment])
pad = pad_sequences(seq, maxlen=512)
model.predict(pad)

comment:Garbage movie with no redeeming qualities. About as subtle as a Michael Bay film, with none of the plot intricacies. Wastes Wanda Maximoff's years of character development. Only one of three movies I've ever wanted to walk out of.


array([[9.9962354e-01, 3.7640490e-04]], dtype=float32)

In [ ]:
layers = [
    # 3001種 * 100 -> 300100
    Embedding(3001, 100, mask_zero=True),
    GlobalAveragePooling1D()
]
partial = Sequential(layers)
partial.summary()

In [32]:
w = model.layers[0].get_weights()
partial.layers[0].set_weights(w)

In [57]:
from scipy.spatial.distance import cosine
word1 = "While this film has a more grim tone than its predecessors it's still a Marvel film at heart. Of course it would take Sam Raimi to find the perfect blend of comic-book-movie, horror, fantasy and slapstick. He might be working with a massive budget now, but the man stays true to his roots and sticking with what he knows pays off.  The pacing may seem disjointed or fragmented at times, but I feel that only reflects the complex nature of the multiverse within the film. Maybe it's not like other groundbreaking MCU films, but ultimately this is an entertaining feature."#@param {type:"string"}
word2 = "This won't be a fan favourite but mixing the marvel cinema experience with a splash of \"old school\" horror hit the target for me.  It's weird, but id go as far as saying that a marvel movie was the best horror flick I've seen in over a decade.  Dark, creepy and visually epic.  The plot was pretty basic, but i had fun watching.  I like that they're mixing it up."#@param {type:"string"}
words = np.array([word1, word2])
seq = tok.texts_to_sequences(words)
pad = pad_sequences(seq)
result = partial.predict(pad)
# cos距離: 越接近1越相似
1 - cosine(result[0], result[1])

0.9721357226371765